<a href="https://colab.research.google.com/github/kjfms/NIA-Project/blob/main/Code/%EC%9E%A5%ED%9D%A5%EA%B5%B0%EC%B2%AD_%EB%B3%B4%EA%B1%B4%EC%86%8C_%EB%AF%BC%EC%9B%90%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 코랩에서 한글 깨질 때 실행
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import gensim

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NIA/11월/청구서전체현황(2020-01-01_2020-12-31).csv',encoding = 'utf-8')

## 변수전처리

In [ ]:
data['접수일'] = pd.to_datetime(data['접수일'],format = '%Y-%m-%d')
data['처리기한'] = pd.to_datetime(data['처리기한'],format = '%Y-%m-%d')
data['처리완료일자'] = pd.to_datetime(data['처리완료일자'],format = '%Y-%m-%d')
data['공개일'] = pd.to_datetime(data['공개일'],format = '%Y-%m-%d')

In [ ]:
# 결측값에 공백넣기 : 명사추출때 코드 돌아가게 하기 위해서
data.fillna({'처리부서':' ','공개내용' : ' '},inplace=True)

In [ ]:
# 부서명 추출
for i in range(len(data)):
  if data.loc[i,'처리부서'] == ' ':
    data.loc[i,'부서명'] = np.nan
  else:
    data.loc[i,'부서명'] = data.loc[i,'처리부서'].split(' ')[2]

In [ ]:
data['부서명'].value_counts()

## 전체 데이터 집계

In [ ]:
# 접수일 월추출
data['접수월'] = data['접수일'].dt.month
# 처리완료일자 월추출
data['처리완료월'] = data['처리완료일자'].dt.month
# 공개일 월추출
data['공개월'] = data['공개일'].dt.month

In [ ]:
# 접수월 별 청구방법 bar-plot
plt.rc('font', family='NanumBarunGothic')

plt.figure(figsize = (10,8))
sns.countplot(x = '접수월',hue = '청구방법', data = data, dodge = False , palette = 'Set3')
plt.title('접수월별 민원 청구방법 그래프')
plt.show()

In [ ]:
# 공개월 별 공개형태 bar-plot
plt.rc('font', family='NanumBarunGothic')

plt.figure(figsize = (10,8))
sns.countplot(x = '공개월',hue = '공개형태', data = data, dodge = False , palette = 'Set3')
plt.title('공개월별 민원 공개형태 그래프')
plt.show()

In [ ]:
# 접수월 별 담당부서 bar-plot
plt.figure(figsize = (30,24))
plt.rc('font', family='NanumBarunGothic')

for i in range(0,12):
  plt.subplot(3,4,i+1)
  pie = data[data['접수월'] == i+1]['부서명'].value_counts()
  pie.plot(kind = 'pie')
  plt.spring()
  plt.xticks(rotation = 70)
  plt.title(i+1)
plt.show()

## Konlpy 설치

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
!apt-get update !apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy 
!JAVA_HOME="C:\Program Files\Java\jdk-16.0.2"

In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

In [ ]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from konlpy.tag import Mecab

from matplotlib import font_manager, rc
import matplotlib.font_manager as fm 
from wordcloud import WordCloud

from collections import Counter

In [ ]:
mecab = Mecab()
okt = Okt()

## 보건소에 해당하는 행만 추출

In [ ]:
# 총 325개
data2 = data[data['부서명'] == '보건소']

In [ ]:
data2.reset_index(drop = True, inplace = True)

## 학습용 데이터 생성

In [ ]:
# 첨부파일이 있는 민원들의 접수번호
contents_null = [6343850,6342663,6347859,6361980, 6360414,6365844,6370809,6376187, 6376127,6375886,6392497, 6408176 ,6411220 ,6424017 ,6423993, 6421206, 6429766, 6433720, 6433613, 6470149,
6477407, 6482634,6486872,6486866,6484838,6496572,6501522,6501474,6504238,6513102,6511367,6511128,6521753,6511115,6533115,6552564, 6564301,6563158, 6568608, 6580512, 6587557, 6592447, 6599511,
6599500, 6599471, 6597479, 6597189, 6645357, 6645357, 6659233, 6663445, 6662997, 6671311, 6670436, 6678598, 6682150, 6696458, 6699212, 6703218, 6717327, 6724342, 6756529, 6772665, 6794693,
6793919, 6817680, 6822431, 6821205, 6833506, 6839221, 6838795, 6843988, 6855427, 6869560, 6907249, 6936585, 7148167, 7288599, 7290360]

In [ ]:
sentence = []

for i in range(len(data2)):
  if data2.loc[i,'접수번호'] in contents_null: # 첨부파일이 있는 민원인 경우
    contents = data2.loc[i,'공개내용'] # 공개내용 가져옴
    title = '' # 타이틀은 안가져옴
  
  else: # 첨부파일이 없는 민원인 경우
    contents = data2.loc[i,'청구내용'] # 청구내용 가져옴
    title = data2.loc[i,'청구제목'] # 청구제목 가져옴

  sentence.append([c for c in okt.nouns(contents) if len(c) >=2 ])
  sentence.append([t for t in okt.nouns(title) if len(t) >=2 ])

## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
nouns_df = pd.DataFrame({'nouns' : sentence})

### 1. stopwords 없이

In [ ]:
tfidf_vectorizer1 = TfidfVectorizer(analyzer='word',
                             lowercase=False,
                             tokenizer=None,
                             preprocessor=None,
                             min_df=5, # 최소 문서 수 5건
                             ngram_range=(1,1), #Bigram : ngram_range=(1,2)
                             smooth_idf=True,
                             )

In [ ]:
tfidf_vector = tfidf_vectorizer1.fit_transform(nouns_df['nouns'].astype('str')) # tf-idf값으로 채워진 DTM생성
tfidf_scores = tfidf_vector.toarray().sum(axis = 0)
tfidf_idx = np.argsort(-tfidf_scores) # 1차원 배열 거꾸로 정렬 의미
tfidf_scores = tfidf_scores[tfidf_idx] # 인덱스
tfidf_vocab = np.array(tfidf_vectorizer1.get_feature_names())[tfidf_idx] # 인덱스에 해당하는 단어 추출


In [ ]:
tfidf_df = pd.DataFrame({'단어' : tfidf_vocab, '점수' : tfidf_scores})

#### 워드클라우드

In [ ]:
tfidf_dict = {}

for i in range(len(tfidf_df)):
  tfidf_dict[tfidf_df.loc[i,'단어']] = tfidf_df.loc[i,'점수']

In [ ]:
plt.figure(figsize = (14,10))
plt.rc('font', family='NanumBarunGothic')

wordcloud = WordCloud(font_path = '/content/drive/MyDrive/Colab Notebooks/졸업논문/BMJUA_TTF.TTF',relative_scaling = 0.2,
                      background_color = 'black', width = 2000,
                      height = 1000,  max_font_size = 200).generate_from_frequencies(tfidf_dict)

plt.imshow(wordcloud)
plt.axis('off')
# plt.title('skip-gram으로 연관성 높은 상위 50개 단어 삭제 후 워드클라우드 ')
plt.show()

### 2. 불용어처리 후

In [ ]:
stopwords = ['아래', '상상', '제안', '까지', '닷컴', '포털', '사이트', '천만', '오아시스', '이벤트', '접수','서울시','서울','특별시',
             '천만상상','파일','첨부','응모','슬로건','공모','공모전','응모전','신청','경우','때문','정도','사항',
                   '해당','겁니다','이것','저것','그것','돋움','신명', '태명', '한컴', '돋움',
                   '동안','거기','저기','여기','대부분','누구','무엇','고딕','만큼','굴림','감사','건지','텐데',
                   '안녕','이번','걸로','수고','겁니까','그간','그건','그때','글쓴이','누가','니다','다면',
                   '뭔가','상상오아시스','하다','이다','되다','같다','궁','자체','서체','정','서','이','을','있다','없다', '체','관련',
                   '생각', '현재', '진행', '사람', '마음', '남산', '내용', '현실','음','막','김','변','조',
                   '오','참','동','지금','주변','대상','부분','요즘','하루','마련','세대','시간','이상','행위',
                   '활동','구분','사실','과정','모습','기간','선정','단지','자신','발생','지역','기대','마련',
                   '장소','모두','부탁','제공','이용','해주','당시','최근','민원','문제','문제점','현황','개선','방안',
                   '문의','답변','일동','요청','담당자','직원','방법','사용','활용','확인','방식','예전']

stopwords.append('자료')
stopwords.append('정보')
stopwords.append('공개')
stopwords.append('청구')
stopwords.append('정보공개')


In [ ]:
tfidf_vectorizer2 = TfidfVectorizer(analyzer='word',
                             lowercase=False,
                             tokenizer=None,
                             preprocessor=None,
                             min_df=5, # 최소 문서 수 5건
                             ngram_range=(1,1), #Bigram : ngram_range=(1,2)
                             smooth_idf=True,
                             stop_words = stopwords
                             )

In [ ]:
tfidf_vector2 = tfidf_vectorizer2.fit_transform(nouns_df['nouns'].astype('str')) # tf-idf값으로 채워진 DTM생성
tfidf_scores2 = tfidf_vector2.toarray().sum(axis = 0)
tfidf_idx2 = np.argsort(-tfidf_scores2) # 1차원 배열 거꾸로 정렬 의미
tfidf_scores2 = tfidf_scores2[tfidf_idx2] # 인덱스
tfidf_vocab2 = np.array(tfidf_vectorizer2.get_feature_names())[tfidf_idx2] # 인덱스에 해당하는 단어 추출


In [ ]:
tfidf_df2 = pd.DataFrame({'단어' : tfidf_vocab2, '점수' : tfidf_scores2})

#### 워드클라우드

In [ ]:
tfidf_dict2 = {}

for i in range(len(tfidf_df2)):
  tfidf_dict2[tfidf_df2.loc[i,'단어']] = tfidf_df2.loc[i,'점수']

In [ ]:
plt.figure(figsize = (14,10))
plt.rc('font', family='NanumBarunGothic')

wordcloud = WordCloud(font_path = '/content/drive/MyDrive/Colab Notebooks/졸업논문/BMJUA_TTF.TTF',relative_scaling = 0.2,
                      background_color = 'white', width = 2000,
                      height = 1000,  max_font_size = 200).generate_from_frequencies(tfidf_dict2)

plt.imshow(wordcloud)
plt.axis('off')
# plt.title('skip-gram으로 연관성 높은 상위 50개 단어 삭제 후 워드클라우드 ')
plt.show()

### 3. 불용어 처리 추가

In [ ]:
stopwords2 = ['아래', '상상', '제안', '까지', '닷컴', '포털', '사이트', '천만', '오아시스', '이벤트', '접수','서울시','서울','특별시',
             '천만상상','파일','첨부','응모','슬로건','공모','공모전','응모전','신청','경우','때문','정도','사항',
                   '해당','겁니다','이것','저것','그것','돋움','신명', '태명', '한컴', '돋움',
                   '동안','거기','저기','여기','대부분','누구','무엇','고딕','만큼','굴림','감사','건지','텐데',
                   '안녕','이번','걸로','수고','겁니까','그간','그건','그때','글쓴이','누가','니다','다면',
                   '뭔가','상상오아시스','하다','이다','되다','같다','궁','자체','서체','정','서','이','을','있다','없다', '체','관련',
                   '생각', '현재', '진행', '사람', '마음', '남산', '내용', '현실','음','막','김','변','조',
                   '오','참','동','지금','주변','대상','부분','요즘','하루','마련','세대','시간','이상','행위',
                   '활동','구분','사실','과정','모습','기간','선정','단지','자신','발생','지역','기대','마련',
                   '장소','모두','부탁','제공','이용','해주','당시','최근','민원','문제','문제점','현황','개선','방안',
                   '문의','답변','일동','요청','담당자','직원','방법','사용','활용','확인','방식','예전','신규','지소','지위','장흥군','통계',
              '상호', '일반','엑셀','대한','업체','안녕하십니까']

stopwords2.append('자료')
stopwords2.append('정보')
stopwords2.append('공개')
stopwords2.append('청구')
stopwords2.append('정보공개')


In [ ]:
tfidf_vectorizer3 = TfidfVectorizer(analyzer='word',
                             lowercase=False,
                             tokenizer=None,
                             preprocessor=None,
                             min_df=5, # 최소 문서 수 5건
                             ngram_range=(1,1), #Bigram : ngram_range=(1,2)
                             smooth_idf=True,
                             stop_words = stopwords2
                             )

In [ ]:
tfidf_vector3 = tfidf_vectorizer3.fit_transform(nouns_df['nouns'].astype('str')) # tf-idf값으로 채워진 DTM생성
tfidf_scores3 = tfidf_vector3.toarray().sum(axis = 0)
tfidf_idx3 = np.argsort(-tfidf_scores3) # 1차원 배열 거꾸로 정렬 의미
tfidf_scores3 = tfidf_scores3[tfidf_idx3] # 인덱스
tfidf_vocab3 = np.array(tfidf_vectorizer3.get_feature_names())[tfidf_idx3] # 인덱스에 해당하는 단어 추출


In [ ]:
tfidf_df3 = pd.DataFrame({'단어' : tfidf_vocab3, '점수' : tfidf_scores3})

#### 워드 클라우드

In [ ]:
tfidf_dict3 = {}

for i in range(len(tfidf_df3)):
  tfidf_dict3[tfidf_df3.loc[i,'단어']] = tfidf_df3.loc[i,'점수']

In [ ]:
plt.figure(figsize = (14,10))
plt.rc('font', family='NanumBarunGothic')

wordcloud = WordCloud(font_path = '/content/drive/MyDrive/Colab Notebooks/졸업논문/BMJUA_TTF.TTF',relative_scaling = 0.2,
                      background_color = 'white', width = 2000,
                      height = 1000,  max_font_size = 150).generate_from_frequencies(tfidf_dict3)

plt.imshow(wordcloud)
plt.axis('off')
# plt.title('skip-gram으로 연관성 높은 상위 50개 단어 삭제 후 워드클라우드 ')
plt.show()

### 4. 불용어 처리 추가
- 기본 70개 + 23 + 5

In [ ]:
stopwords3 = ['아래', '상상', '제안', '까지', '닷컴', '포털', '사이트', '천만', '오아시스', '이벤트', '접수','서울시','서울','특별시',
             '천만상상','파일','첨부','응모','슬로건','공모','공모전','응모전','신청','경우','때문','정도','사항',
                   '해당','겁니다','이것','저것','그것','돋움','신명', '태명', '한컴', '돋움',
                   '동안','거기','저기','여기','대부분','누구','무엇','고딕','만큼','굴림','감사','건지','텐데',
                   '안녕','이번','걸로','수고','겁니까','그간','그건','그때','글쓴이','누가','니다','다면',
                   '뭔가','상상오아시스','하다','이다','되다','같다','궁','자체','서체','정','서','이','을','있다','없다', '체','관련',
                   '생각', '현재', '진행', '사람', '마음', '남산', '내용', '현실','음','막','김','변','조',
                   '오','참','동','지금','주변','대상','부분','요즘','하루','마련','세대','시간','이상','행위',
                   '활동','구분','사실','과정','모습','기간','선정','단지','자신','발생','지역','기대','마련',
                   '장소','모두','부탁','제공','이용','해주','당시','최근','민원','문제','문제점','현황','개선','방안',
                   '문의','답변','일동','요청','담당자','직원','방법','사용','활용','확인','방식','예전','신규','지소','지위','장흥군','통계',
              '상호', '일반','엑셀','대한','업체','안녕하십니까','주소','포함','처리','전화번호','연락처','리스트','오픈','등록','산보','원외','사별','보건']

stopwords3.append('자료')
stopwords3.append('정보')
stopwords3.append('공개')
stopwords3.append('청구')
stopwords3.append('정보공개')


In [ ]:
tfidf_vectorizer4 = TfidfVectorizer(analyzer='word',
                             lowercase=False,
                             tokenizer=None,
                             preprocessor=None,
                             min_df=5, # 최소 문서 수 5건
                             ngram_range=(1,1), #Bigram : ngram_range=(1,2)
                             smooth_idf=True,
                             stop_words = stopwords3
                             )

In [ ]:
tfidf_vector4 = tfidf_vectorizer4.fit_transform(nouns_df['nouns'].astype('str')) # tf-idf값으로 채워진 DTM생성
tfidf_scores4 = tfidf_vector4.toarray().sum(axis = 0)
tfidf_idx4 = np.argsort(-tfidf_scores4) # 1차원 배열 거꾸로 정렬 의미
tfidf_scores4 = tfidf_scores4[tfidf_idx4] # 인덱스
tfidf_vocab4 = np.array(tfidf_vectorizer4.get_feature_names())[tfidf_idx4] # 인덱스에 해당하는 단어 추출


In [ ]:
tfidf_df4 = pd.DataFrame({'단어' : tfidf_vocab4, '점수' : tfidf_scores4})

#### 워드클라우드

In [ ]:
tfidf_dict4 = {}

for i in range(len(tfidf_df4)):
  tfidf_dict4[tfidf_df4.loc[i,'단어']] = tfidf_df4.loc[i,'점수']

In [ ]:
plt.figure(figsize = (14,10))
plt.rc('font', family='NanumBarunGothic')
plt.rc('font', size = 12)

wordcloud = WordCloud(font_path = '/content/drive/MyDrive/Colab Notebooks/졸업논문/BMJUA_TTF.TTF',relative_scaling = 0.2,
                      background_color = 'white', width = 2000,
                      height = 1000,  max_font_size = 200).generate_from_frequencies(tfidf_dict4)

plt.imshow(wordcloud)
plt.axis('off')
plt.title('장흥군청 보건소 TF-IDF 점수 별 키워드 시각화')
plt.show()

- 전체 TF-IDF 에서 선정한 키워드 제외, 점수가 높지만 유의미해 보이지 않는 단어의 경우 임의적으로 불용어로 처리해줌

#### 상위 5개 키워드 살피기

##### 각 키워드의 추이 살피기

## 빈도별 키워드 추출
- TF-IDF와는 불용어 처리 다르게함

### 1차 : 불용어 처리 X

In [ ]:
sentence2 = [[] for i in range(len(sentence))]

for t1 in range(len(sentence)):
  for t2 in sentence[t1]:
    if t2 not in stopwords3:
      sentence2[t1].append(t2)

In [ ]:
all_nouns = []

for t1 in range(len(sentence2)):
  for t2 in sentence2[t1]:
    all_nouns.append(t2)

In [ ]:
all_nouns_set = list(set(all_nouns))

In [ ]:
# 딕셔너리 생성
all_nouns_dict = dict.fromkeys(all_nouns_set,0)

In [ ]:
# 빈도수 카운트 : 청구제목, 청구내용, 공개내용에 한번이라도 들어있으면 OK

for word in all_nouns_dict.keys():
  for i in range(len(data2)):
    if word in data2.loc[i,'청구제목'] or word in data2.loc[i,'청구내용'] or word in data2.loc[i,'공개내용']:
      all_nouns_dict[word] += 1
    else:
      pass


In [ ]:
sorted_all_nouns_dict = dict(sorted(all_nouns_dict.items(), key = lambda item: item[1], reverse = True))

#### 워드클라우드

In [ ]:
plt.figure(figsize = (14,10))
plt.rc('font', family='NanumBarunGothic')

wordcloud = WordCloud(font_path = '/content/drive/MyDrive/Colab Notebooks/졸업논문/BMJUA_TTF.TTF',relative_scaling = 0.2,
                      background_color = 'black', width = 2000,
                      height = 1000,  max_font_size = 200,colormap = 'seismic').generate_from_frequencies(sorted_all_nouns_dict)

plt.imshow(wordcloud)
plt.axis('off')
plt.title('장흥군 보건소 빈도별 키워드 워드클라우드')
plt.show()

### 1차 불용어 처리후 진행
- stopword 

In [ ]:
all_nouns_set2 = [word for word in all_nouns_set if word not in stopwords]

In [ ]:
# 딕셔너리 생성
all_nouns_dict2 = dict.fromkeys(all_nouns_set2,0)

# 빈도수 카운트 : 청구제목, 청구내용, 공개내용에 한번이라도 들어있으면 OK

for word in all_nouns_dict2.keys():
  for i in range(len(data2)):
    if word in data2.loc[i,'청구제목'] or word in data2.loc[i,'청구내용'] or word in data2.loc[i,'공개내용']:
      all_nouns_dict2[word] += 1
    else:
      pass

sorted_all_nouns_dict2 = dict(sorted(all_nouns_dict2.items(), key = lambda item: item[1], reverse = True))

In [ ]:
sorted_all_nouns_df2 = pd.DataFrame({'단어': sorted_all_nouns_dict2.keys(), '빈도': sorted_all_nouns_dict2.values()})

### 2차 불용어 처리후 진행
- stopword + '전화' ,'장흥', '번호', '일로', '부터', '주시', '중위', '관내', '일부', '제외', '연락','작일','시작','메일','전체','오름','개인','부존재

In [ ]:
stopwords4 = ['아래', '상상', '제안', '까지', '닷컴', '포털', '사이트', '천만', '오아시스', '이벤트', '접수','서울시','서울','특별시',
             '천만상상','파일','첨부','응모','슬로건','공모','공모전','응모전','신청','경우','때문','정도','사항',
                   '해당','겁니다','이것','저것','그것','돋움','신명', '태명', '한컴', '돋움',
                   '동안','거기','저기','여기','대부분','누구','무엇','고딕','만큼','굴림','감사','건지','텐데',
                   '안녕','이번','걸로','수고','겁니까','그간','그건','그때','글쓴이','누가','니다','다면',
                   '뭔가','상상오아시스','하다','이다','되다','같다','궁','자체','서체','정','서','이','을','있다','없다', '체','관련',
                   '생각', '현재', '진행', '사람', '마음', '남산', '내용', '현실','음','막','김','변','조',
                   '오','참','동','지금','주변','대상','부분','요즘','하루','마련','세대','시간','이상','행위',
                   '활동','구분','사실','과정','모습','기간','선정','단지','자신','발생','지역','기대','마련',
                   '장소','모두','부탁','제공','이용','해주','당시','최근','민원','문제','문제점','현황','개선','방안',
                   '문의','답변','일동','요청','담당자','직원','방법','사용','활용','확인','방식','예전','자료','정보','공개','청구','정보공개',
              '전화' ,'장흥', '번호', '일로', '부터', '주시', '중위', '관내', '일부', '제외', '연락','작일','시작','메일','전체','오름','개인','부존재']

In [ ]:
all_nouns_set3 = [word for word in all_nouns_set if word not in stopwords4]

In [ ]:
# 딕셔너리 생성
all_nouns_dict3 = dict.fromkeys(all_nouns_set3,0)

# 빈도수 카운트 : 청구제목, 청구내용, 공개내용에 한번이라도 들어있으면 OK

for word in all_nouns_dict3.keys():
  for i in range(len(data2)):
    if word in data2.loc[i,'청구제목'] or word in data2.loc[i,'청구내용'] or word in data2.loc[i,'공개내용']:
      all_nouns_dict3[word] += 1
    else:
      pass

sorted_all_nouns_dict3 = dict(sorted(all_nouns_dict3.items(), key = lambda item: item[1], reverse = True))

In [ ]:
plt.figure(figsize = (14,10))
plt.rc('font', family='NanumBarunGothic')

wordcloud = WordCloud(font_path = '/content/drive/MyDrive/Colab Notebooks/졸업논문/BMJUA_TTF.TTF',relative_scaling = 0.2,
                      background_color = 'white', width = 2000,
                      height = 1000,  max_font_size = 200).generate_from_frequencies(sorted_all_nouns_dict3)

plt.imshow(wordcloud)
plt.axis('off')
plt.rc('font', size=12)
plt.title('장흥군청 보건소 빈도별 키워드 시각화')
plt.show()